## Numerical Analysis - Fall semester 2024
# Serie 04 - Variants of Newton's method and Polynomial Interpolation

As usual, we will import some useful packages for later reference. You will have to run this cell every time you restart your notebook.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

<hr style="clear:both">

### Newton's method for systems of equations

<div class="alert alert-success">
    
**Exercise 1:** Complete the following implementation of Newton's method for systems of equations (Algorithm 1.7 in the lecture notes).

*Hint:* To solve an equation of the form $Ax = b$ for $x$, where $A \in \mathbb{R}^{n \times n}$, $b \in \mathbb{R}^{n}$, and $x \in \mathbb{R}^{n}$, use `x = np.linalg.solve(A, b)`. To compute the norm $\lVert x \rVert$ of a vector $x$, use `np.linalg.norm(x)`.
</div>


In [ ]:
def newtonsys(F, JF, x0, tol, nmax):
    x = []  # list of iterates
    x.append(x0)
    r = []  # list of increments
    r.append(tol + 1)
    k = 0  # iteration counter
    while r[-1] > tol and k < nmax:
        ### BEGIN SOLUTION
        dx = np.linalg.solve(JF(x[-1]), -F(x[-1]))
        x.append(x[-1] + dx)
        r.append(np.linalg.norm(x[-1] - x[-2]))
        k = k + 1
        ### END SOLUTION
    return x, r, k

Let us consider the nonlinear system $\mathbf{F}({\bf x})=\mathbf{0}$ with

$$
\mathbf{F}({\bf x})
%F(x_1,x_2)
=\left[
\begin{array}{lcl}
e^{x_1^2+x_2^2}-\alpha \\
e^{x_1^2-x_2^2}-1
\end{array}
\right],
$$
where the parameter $\alpha$ takes the values $1$ or $e$.

<div class="alert alert-success">
    
**Exercise 2 (Theoretical):** Compute the Jacobian matrix $J_\mathbf{F}$ associated to the nonlinear system and the first Newton iteration in the case $\alpha=e$ with ${\bf x}^{(0)}=(1,1)^\top$ as initial point.
</div>

=== BEGIN MARK SCHEME ===

We have
$$
J_\mathbf{F}({\bf x})=
\left[
\begin{array}{lc}
2 x_1 e^{x_1^2 + x_2^2}  &   2 x_2 e^{x_1^2 + x_2^2}  \\
2 x_1 e^{x_1^2 - x_2^2}  &  - 2 x_2 e^{x_1^2 - x_2^2}
\end{array}
\right].
$$

The first Newton iteration ${\bf x}^{(1)}$ is given by ${\bf x}^{(1)}={\bf x}^{(0)}+\delta{\bf x}$, where $\delta{\bf x}$ is the solution of

$$
J_{\bf F}({\bf x}^{(0)})\delta{\bf x}=-\mathbf{F}({\bf x}^{(0)}).
$$

For ${\bf x}^{(0)}=(1,1)^\top$ we have

$${\bf F}({\bf x}^{(0)})=\left[
\begin{array}{c}
e(e-1) \\
0
\end{array}
\right]
\quad \text{and} \quad
J_{\bf F}({\bf x}^{(0)})=\left[
\begin{array}{lc}
2e^2  &  2e^2  \\
2  &  - 2
\end{array}
\right].
$$

This means we have to solve the system

$$
\left[
\begin{array}{lc}
2e^2  &  2e^2  \\
2  &  - 2
\end{array}
\right]\left[
\begin{array}{c}
\delta x_1 \\
\delta x_2
\end{array}
\right]=\left[
\begin{array}{c}
e(1-e) \\
0
\end{array}
\right].
$$

The second equation gives us $\delta x_1=\delta x_2$ and, replacing this result in the first one, we get $\delta x_1=\frac{1-e}{4e}$, namely $\delta{\bf x}=\frac{1-e}{4e}{\bf x}^{(0)}$. Finally, we find ${\bf x}^{(1)}=(\frac{1-e}{4e}+1,\frac{1-e}{4e}+1)^\top\approx (0.84197,0.84197)^\top$.

=== END MARK SCHEME ===

<div class="alert alert-success">

**Exercise 3:** Fix a tolerance of `tol = 1e-8` and a maximal number of iterations `nmax=1000`. Apply the vector Newton method for both cases $\alpha=1$ and $\alpha=e$, using as initial value ${\bf x}_1^{(0)}=(1/10,1/10)^\top$ and ${\bf x}_e^{(0)}=(1,1)^\top$, respectively. How do the result and the number of iterations change? What is the order of convergence of the method in each case?

*Hint:* Try to work as much with NumPy arrays as possible, since you can do mathematical operations with them (`+`, `-`, ...), which are not possible between lists. To convert a Python list `l = [1, 2, 3]` to a NumPy array, use `np.array(l)`.
</div>

<div class="alert alert-info">

**Note:** Choosing the initial point can strongly influence if, and to which zero, Newton's method converges. In general, the closer to the zero the initial point is chosen, the better the method will converge to that zero. For one-dimentional functions with one input, we can often visually determine a good starting point. Once we move to two or more  inputs, we may need to use a few iterations of another method (bisection, ...) to find a suitable stating point.
</div>

In [ ]:
def F(x, alpha):
    ### BEGIN SOLUTION
    return np.array([np.exp(x[0] ** 2 + x[1] ** 2) - alpha, np.exp(x[0] ** 2 - x[1] ** 2) - 1])
    ### END SOLUTION

def JF(x):
    ### BEGIN SOLUTION
    return np.array([[2 * x[0] * np.exp(x[0] ** 2 + x[1] ** 2),
                     2 * x[1] * np.exp(x[0] ** 2 + x[1] ** 2)],
                    [2 * x[0] * np.exp(x[0] ** 2 - x[1] ** 2),
                     -2 * x[1] * np.exp(x[0] ** 2 - x[1] ** 2)]])
    ### END SOLUTION

# the function F when α = 1
def F_1(x):
    return F(x, alpha=1)

# the function F when α = e
def F_e(x):
    return F(x, alpha=np.e)

### BEGIN SOLUTION
x0_1 = [1/10, 1/10]
x0_e = [1, 1]

tol = 1e-8
nmax = 1000

x_1, r_1, niter_1 = newtonsys(F_1, JF, x0_1, tol, nmax)
x_e, r_e, niter_e = newtonsys(F_e, JF, x0_e, tol, nmax)

print(f"converged to x_1 = {x_1[-1]} after {niter_1} iterations")
print(f"converged to x_e = {x_e[-1]} after {niter_e} iterations")

plt.semilogy(range(niter_1), np.array(r_1[1:]) / np.array(r_1[:-1]), marker="o", label=r"$p=1$")
plt.semilogy(range(niter_1), np.array(r_1[1:]) / np.array(r_1[:-1]) ** 2, marker="o", label=r"$p=2$")
plt.ylabel(r"increment ratio $\|| x^{(k + 1)} - x^{(k)} \|| / \|| x^{(k)} - x^{(k - 1)} \||^{p}$")
plt.xlabel(r"iteration $k$")
plt.legend()
plt.grid()
plt.show()
plt.semilogy(range(niter_e), np.array(r_e[1:]) / np.array(r_e[:-1]), marker="o", label=r"$p=1$")
plt.semilogy(range(niter_e), np.array(r_e[1:]) / np.array(r_e[:-1]) ** 2, marker="o", label=r"$p=2$")
plt.ylabel(r"increment ratio $\|| x^{(k + 1)} - x^{(k)} \|| / \|| x^{(k)} - x^{(k - 1)} \||^{p}$")
plt.xlabel(r"iteration $k$")
plt.legend()
plt.grid()
plt.show()

# We see that the Newton method converges with second order in the case α = e
# whereas in the case α = 1, it only converges with first order. This is due
# to the fact that det(Jf([0, 0])) = 0.

### END SOLUTION

<hr style="clear:both">

### Damped Newton's method

We provide you with the implementation of Newton's method to find the zero of a function $g$ which you've completed in last week's notebook. However, there is one slight modification: we introduce a damping factor $0 < \gamma \leq 1$ in the iteration

\begin{equation*}
     x^{(k + 1)} = x^{(k)} - \gamma \frac{f(x^{(k)})}{f'(x^{(k)})}.
\end{equation*}

For $\gamma = 1$ we end up with the standard Newton's method.

<div class="alert alert-info">

**Note:** Damping is a technique that reduces the size of the Newton step to ensure more stable behavior, especially in cases where the method without damping might oscillate too much or fail to converge. Instead of taking the full Newton step, the damped method takes only a portion of the step, usually scaled by a factor $\gamma_k$, where $0 < \gamma_k \leq 1$, which changes in every iteration $k$. The advantage is that it's more stable in cases where derivatives exhibit large oscillations or when the system is poorly conditioned. For simplicity, we only consider a fixed damping $\gamma_k = \gamma$ for all iterations $k$ in our implementation.
</div>

In [ ]:
def newton_damped(f, df, x0, tol, nmax, gamma):
    x = []  # list of iterates
    x.append(x0)
    r = []  # list of increments
    r.append(tol + 1)
    k = 0  # iteration counter
    while r[-1] > tol and k < nmax:
        x.append(x[-1] - gamma * f(x[-1]) / df(x[-1]))
        r.append(abs(x[-1] - x[-2]))
        k = k + 1
    return x, r, k

Further, we provide you with a Python function which, given the function $f$ on which the Newton iteration $x = [x^{(0)}, x^{(1)}, \dots, x^{(k)}]$ has been computed, visualizes the iteration in a plot.

In [ ]:
def plot_iteration(f, x):
    x = np.asarray(x)
    a = np.min(x) - 1
    b = np.max(x) + 1
    x_lin = np.linspace(a, b, 100)
    plt.plot(x_lin, f(x_lin), label="$f(x)$")
    plt.plot(x, f(x), marker="o", label="iteration")
    plt.xlabel(r"$x$")
    plt.legend()
    plt.grid()
    plt.show()

The plot shows the function $f$, as well as the iterates represented with the points $(x^{(k)}, f(x^{(k)})), k=0, 1, \dots$ along the function $f$. These points are additionally connected with a line, to emphasize the order in which they were obtained by the algorithm.

<div class="alert alert-success">

**Exercise 4:** For the function $f(x) = 1 - e^{-x^2}$, run the damped Newton's method from the starting point $x_0 = 1.5$ on a tolerance of $10^{-5}$ for at most $1000$ iterations. Play around with the values of the damping parameter $0 < \gamma \leq 1$, and observe what effect it has on the convergence. Can you find a damping $\gamma$, for which the method converges in less iterations than the standard method ($\gamma = 1$)?
</div>

In [ ]:
### BEGIN SOLUTION
def f(x):
    return 1 - np.exp(- x**2)

def df(x):
    return 2 * x * np.exp(- x**2)

x0 = 1.5
tol = 1e-5
nmax = 1000

# Standard Newton's method
gamma = 1.0
x, r, k = newton_damped(f, df, x0, tol, nmax, gamma)
plot_iteration(f, x)
print(f"converged in {k} iterations")
# Observation: Since the derivative at the starting point is comparatively small, the first step size is very big.
# In fact, we jump to the other side of the zero, which could have lead to certain issues.

# Very strong damping
gamma = 0.1
x, r, k = newton_damped(f, df, x0, tol, nmax, gamma)
plot_iteration(f, x)
print(f"converged in {k} iterations")
# Observation: A strong damping leads to a very stable convergence. However, many more iterations are needed.

# Damping for which it takes less iterations
gamma = 0.94
x, r, k = newton_damped(f, df, x0, tol, nmax, gamma)
plot_iteration(f, x)
print(f"converged in {k} iterations")
# Observation: Usually, the damped Newton's method converges slower. The fact that we could find a parameter γ for 
# which the method converges faster is more of a lucky coincidence, and not usually exploited in practice.
### END SOLUTION

<hr style="clear:both">

### Simple polynomial interpolation

We have the following data

\begin{align*}
x_1=0 \quad  & y_1=12, \\
x_2=1 \quad  & y_2=18, \\
x_3=2 \quad  & y_3=6.
\end{align*}

<div class="alert alert-success">

**Exercise 5:** Find the second order polynomial interpolating the data by setting up and solving the linear system with the Vandermonde matrix.
</div>

In [ ]:
### BEGIN SOLUTION
V = np.array([[1, 0, 0],
              [1, 1, 1],
              [1, 2, 4]])

y = np.array([12, 18, 6])
a = np.linalg.solve(V, y)
print(f"the interpolating polynomial is p₂(x) = {a[0]} + {a[1]} * x + {a[2]} * x^2")
### END SOLUTION

<div class="alert alert-success">

**Exercise 6:** Find the second order polynomial interpolating the data by using `np.polyfit`, evaluate the resulting polynomial at $100$ evenly spaced points in the interval $[-1, 3]$ by using `np.polyval`, and use them to visualize the polynomial on the same plot with the data.

*Hint:* To plot individual data points, use `plt.scatter`. To find out more about a function's inputs and outputs, use `help(function_name)`.
</div>

In [ ]:
### BEGIN SOLUTION
x = np.array([0, 1, 2])
a = np.polyfit(x, y, 2)
x_lin = np.linspace(-1, 3, 100)
y_lin = np.polyval(a, x_lin)
plt.scatter(x, y, label=r"data $(x_i, y_i)$")
plt.plot(x_lin, y_lin, label=r"interpolant $p_2(x)$")
plt.grid()
plt.ylabel(r"$y$")
plt.xlabel(r"$x$")
plt.legend()
plt.show()
### END SOLUTION

<div class="alert alert-success">

**Exercise 7 (Theoretical):** Find the three polynomials of the Lagrangian basis corresponding to the points $x_1=0, x_2=1$ and $x_3=2$.

*Hint:* Use the formula given in Definition 2.2 of the lecture notes.
</div>

=== BEGIN MARK SCHEME ===

Using the formula from Definition 2.2

\begin{align*}
\phi_i(x)=\prod_{\stackrel{j=1}{j\neq i}}^3 \frac{x-x_j}{x_i-x_j}, \quad \quad i=1,2,3.
\end{align*}

we find the Lagrangian polynomials by replacing the values of $x_1 = 0$, $x_2 = 1$, and $x_3 = 2$ to get

\begin{align*}
& \phi_1(x)=\frac{x-x_2}{x_1-x_2}\cdot \frac{x-x_3}{x_1-x_3}= \frac{(x-1)(x-2)}{2}\\
& \phi_2(x)=\frac{x-x_1}{x_2-x_1}\cdot \frac{x-x_3}{x_2-x_3}= -x(x-2)\\
& \phi_3(x)=\frac{x-x_1}{x_3-x_1}\cdot \frac{x-x_2}{x_3-x_2}= \frac{x(x-1)}{2}.  
\end{align*}

=== END MARK SCHEME ===

<div class="alert alert-success">

**Exercise 8 (Theoretical):** Using your result from Exercise 7, find the second order polynomial interpolating the data. Compare the resulting polynomial to what you've found in Exercise 5.
</div>

=== BEGIN MARK SCHEME ===

The sought polynomial is given by

$$
p_2(x)= y_1 \phi_1(x) + y_2 \phi_2(x) + y_3 \phi_3(x).
$$

Inserting the Lagrange polynomials from Exercise 7 and the $y$-data, we get

\begin{align*}
p_2(x)=& 6(x-1)(x-2)-18 x(x-2)+3x(x-1)\\
=& -9 x^2 + 15 x + 12.
\end{align*}

This is the same polynomial as we've found in Exercise 5.

=== END MARK SCHEME ===

<hr style="clear:both">

### Interpolating functions at uniform and non-uniform nodes

We are given the data $(x_i,y_i)$, $i=1,\ldots,n+1$, where the $x_i$ are $n+1$ nodes in the interval $[a,b]=[-5,5]$ and
the values $y_i$ come from evaluation of the function
$$
  g(x)= \frac{1}{1+\exp(4x)}
$$
without any measurement error, namely $y_i = g(x_i)$.  We want
to apply polynomial interpolation to the function $g$ from the data $(x_i,y_i)$.

<div class="alert alert-success">

**Exercise 9:** For degrees $n=6$ and $n=14$, compute the polynomial $p_n$ interpolating the data $(x_i,y_i)$, $i=1,...,n+1$ with uniformly spaced nodes

\begin{equation*}
    x_i = a + \frac{i - 1}{n} (b - a), ~i=1,...,n+1.
\end{equation*}

Visualize the function and the interpolating polynomial at $100$ evenly spaced points in the interval $[-5, 5]$.
</div>

In [ ]:
### BEGIN SOLUTION
def g(x):
    return 1 / (1 + np.exp(4 * x))

x_lin = np.linspace(-5, 5, 100)
y_lin = g(x_lin)

x_6_unif = - 5 + np.arange(6 + 1) / 6 * (5 - (- 5))
y_6_unif = g(x_6_unif)
a_6_unif = np.polyfit(x_6_unif, y_6_unif, 6)
y_lin_6_unif = np.polyval(a_6_unif, x_lin)

x_14_unif = - 5 + np.arange(14 + 1) / 14 * (5 - (- 5))
y_14_unif = g(x_14_unif)
a_14_unif = np.polyfit(x_14_unif, y_14_unif, 14)
y_lin_14_unif = np.polyval(a_14_unif, x_lin)

plt.plot(x_lin, y_lin, label=r"function $g(x)$")
plt.scatter(x_6_unif, y_6_unif, label=r"$(x_i, y_i)$ for $n = 6$")
plt.plot(x_lin, y_lin_6_unif, label=r"interpolant $p_6(x)$")
plt.scatter(x_14_unif, y_14_unif, label=r"$(x_i, y_i)$ for $n = 14$")
plt.plot(x_lin, y_lin_14_unif, label=r"interpolant $p_{14}(x)$")
plt.grid()
plt.ylabel(r"$y$")
plt.xlabel(r"$x$")
plt.legend()
plt.show()
### END SOLUTION

<div class="alert alert-success">

**Exercise 10:** Repeat Exercise 9 for Clenshaw-Curtis nodes, i.e. nodes which are given by

\begin{equation*}
    x_i = \frac{a + b}{2} - \frac{b - a}{2} \cos\left( \frac{\pi(i - 1)}{n} \right),  ~i=1,...,n+1.
\end{equation*}

Compare the result to Exercise 10. Explain the difference using what you've learned in the lecture.

</div>

In [ ]:
### BEGIN SOLUTION
x_6_cc = (5 + (- 5)) / 2 + (5 - (- 5)) / 2 * np.cos(np.pi * np.arange(6 + 1) / 6) 
y_6_cc = g(x_6_cc)
a_6_cc = np.polyfit(x_6_cc, y_6_cc, 6)
y_lin_6_cc = np.polyval(a_6_cc, x_lin)

x_14_cc = (5 + (- 5)) / 2 + (5 - (- 5)) / 2 * np.cos(np.pi * np.arange(14 + 1) / 14) 
y_14_cc = g(x_14_cc)
a_14_cc = np.polyfit(x_14_cc, y_14_cc, 14)
y_lin_14_cc = np.polyval(a_14_cc, x_lin)

plt.plot(x_lin, y_lin, label=r"function $g(x)$")
plt.scatter(x_6_cc, y_6_cc, label=r"$(x_i, y_i)$ for $n = 6$")
plt.plot(x_lin, y_lin_6_cc, label=r"interpolant $p_6(x)$")
plt.scatter(x_14_cc, y_14_cc, label=r"$(x_i, y_i)$ for $n = 14$")
plt.plot(x_lin, y_lin_14_cc, label=r"interpolant $p_{14}(x)$")
plt.grid()
plt.ylabel(r"$y$")
plt.xlabel(r"$x$")
plt.legend()
plt.show()

# Difference: Much better approximation for Clenshaw-Curti's nodes.
# Explanation: Clenshaw-Curtis nodes have a slower growing Lebesgue constant than uniformly spaced nodes.
# Therefore, by Definition 2.4, the interpolation is much more stable.

### END SOLUTION

<hr style="clear:both">

## The end

Amazing! You have reached the end of the fourth exercise notebook. 